In [1]:
!pip install chromadb

  Obtaining dependency information for chromadb from https://files.pythonhosted.org/packages/d2/24/1141557c77cf99e01baf16370ede5c0d43bef2fe17b36b8851bafd021336/chromadb-0.4.21-py3-none-any.whl.metadata
  Obtaining dependency information for chroma-hnswlib==0.7.3 from https://files.pythonhosted.org/packages/d2/32/a91850c7aa8a34f61838913155103808fe90da6f1ea4302731b59e9ba6f2/chroma_hnswlib-0.7.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for fastapi>=0.95.2 from https://files.pythonhosted.org/packages/ad/b9/b7ea33663daffa9db94119ea2a3df8f97bdca297024145fe79a5a13d37af/fastapi-0.105.0-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn[standard]>=0.18.3 from https://files.pythonhosted.org/packages/26/59/fddd9df489fe27f492cc97626e03663fb3b9b6ef7ce8597a7cdc5f2cbbad/uvicorn-0.25.0-py3-none-any.whl.metadata
  Obtaining dependency information for posthog>=2.4.0 from https://files.pythonhosted.org/packages/3b/82/441cb77a43499661228048dcd0d21e0ae3235

In [2]:
import chromadb
from chromadb.utils import embedding_functions

CHROMA_DATA_PATH = "chroma_data/"
EMBED_MODEL = "all-MiniLM-L6-v2"
COLLECTION_NAME = "sample_docs"
client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)

In [4]:
embeddingfunction = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)

collection = client.create_collection(name=COLLECTION_NAME,embedding_function=embeddingfunction,
                                      metadata={"hnsw:space": "cosine"})

In [7]:
documents = [
"Operating the Climate Control System  Your car has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. ",
"The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed",
"The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car.",
"The car will blow warm air into the car.",
"The car will blow warm air onto the windshield to defrost it."

]

topics = ["control system","airflow","mode","heat","Defrost"]



In [8]:
collection.add(documents=documents,
ids=[f"id{i}" for i in range(len(documents))],
metadatas=[{"topic": t} for t in topics])

In [9]:
query_results = collection.query(query_texts=["what are the available modes"], n_results=1,)

In [10]:
query_results.keys()

dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])

In [11]:
query_results["documents"]

[['The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car.']]

In [12]:
query_results["ids"]

[['id2']]

In [13]:
query_results["distances"]

[[0.40187349500535396]]

In [14]:
query_results["metadatas"]

[[{'topic': 'mode'}]]

In [15]:
query_results = collection.query(query_texts=["what are the available modes"],
                                 where={"topic": {"$eq": "airflow"}},n_results=1,)

In [16]:
query_results["documents"]

[['The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed']]

In [17]:
query_results["distances"]

[[0.9115099883580479]]